## Laboratorio 1 Data Science

# Stefano Aragoni, Carol Arevalo

In [559]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [560]:
datos_mnist, info_mnist = tfds.load(name='mnist',
                                    shuffle_files = False,
                                    with_info=True, 
                                    as_supervised=True)

## Datos


In [561]:
entreno_mnist, prueba_mnist = datos_mnist['train'], datos_mnist['test']

In [562]:
num_obs_validacion = 0.1 * info_mnist.splits['train'].num_examples

In [563]:
num_obs_validacion = tf.cast(num_obs_validacion, tf.int64)

In [564]:
num_obs_prueba = info_mnist.splits['test'].num_examples

In [565]:
num_obs_prueba = tf.cast(num_obs_prueba, tf.int64)

In [566]:
def normalizar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32)
    imagen /= 255.
    return imagen, etiqueta

In [567]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)

In [568]:
datos_prueba = prueba_mnist.map(normalizar)

In [569]:
TAMANIO_BUFFER = 10000

In [570]:
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)

In [571]:
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)

In [572]:
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [573]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)

In [574]:
entradas_validacion, metas_validacion = next(iter(datos_validacion))

## Modelo

In [575]:
tamanio_entrada = 784
tamanio_salida = 10

Tamaño Orginal

In [576]:
tamanio_capa_escondida = 50

Tamaño de 200

In [577]:
# tamanio_capa_escondida = 200

Tamaño de 100

In [578]:
# tamanio_capa_escondida = 100

Modelo Original

In [579]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

Modelo con 3 capas escondida


In [580]:
# modelo = tf.keras.Sequential([

#     tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    
#     tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
# ])

Modelo con 4 capas escondida


In [581]:
# modelo = tf.keras.Sequential([

#     tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4ta capa escondida

#     tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
# ])

Modelo utilizando Sigmoid

In [582]:
# modelo = tf.keras.Sequential([

#     tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida


#     tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
# ])

Modelo usando Relu y tahn

In [583]:
# modelo = tf.keras.Sequential([

#     tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
#     tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    
#     tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
# ])

In [584]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [585]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 6s - loss: 0.4115 - accuracy: 0.8870 - val_loss: 0.2124 - val_accuracy: 0.9378 - 6s/epoch - 11ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1771 - accuracy: 0.9474 - val_loss: 0.1625 - val_accuracy: 0.9527 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 4s - loss: 0.1300 - accuracy: 0.9610 - val_loss: 0.1210 - val_accuracy: 0.9665 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 3s - loss: 0.1042 - accuracy: 0.9684 - val_loss: 0.0997 - val_accuracy: 0.9702 - 3s/epoch - 6ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0845 - accuracy: 0.9745 - val_loss: 0.0883 - val_accuracy: 0.9738 - 3s/epoch - 6ms/step


In [586]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 1s 672ms/step - loss: 0.0979 - accuracy: 0.9702


In [587]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.10. Precisión de prueba: 97.02%
